<center><h1>Corpus to Conclusions:</h1></center>
<center><h4>Integrating Full NLP Pipeline Techniques For Climate Change Text Analytics</h4></center>

By: **Elijah Taber**

In [3]:
# Data Manipulation
import pandas as pd
import numpy as np
import random
import pickle
import warnings
import umap
import sqlite3

warnings.filterwarnings("ignore")

# Machine Learning
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Deep Learning
import tensorflow as tf
import torch
import tensorflow_hub as hub
from torch.utils.data import DataLoader

# NLP
import transformers
import spacy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic import BERTopic
from transformers import (
    RobertaTokenizer, 
    RobertaModel, 
    BartTokenizer, 
    BartModel
)

nlp = spacy.load('en_core_web_sm')

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

# Custom Data Processors
from src.normalizer import TextNormalizer
from src.pdf_processor import pdf_text_extractor

#########################################################################################
#                           Text Classification Network (C4NN)                          #
#########################################################################################
from nlp.\
        text_classification_network.\
            classification_cnn import (
                CNNHyperModel as C4NN, 
                BayesianTuner
)
from nlp.\
        text_classification_network.\
            classification_setup import (
                LexicalTokenizer, 
                EmbeddingMatrix
)
from nlp.\
        text_classification_network.\
            classification_visualizer import (
                SphericalMap
)
from nlp.\
        text_classification_network.\
            classification_evaluator import (
                C4NNEvaluator
)

#########################################################################################
#                            Topic Transformer (CLIMATopic)                             #
#########################################################################################
from nlp.\
        topic_model.\
            topic_transformer import (
                CLIMATopic
)
from nlp.\
        topic_model.\
            tm_config import (
                BERTopicConfig, 
                SaveConfig, 
                VisualizationConfig,
                EvaluationConfig
)
from nlp.\
        topic_model.\
            topic_visualizations import (
                visualize_umap,
                visualize_topics, 
                visualize_heatmap, 
                visualize_document_datamap
)
from nlp.\
        topic_model.\
            bertopic_setup import (
                create_umap
)
from nlp.\
        topic_model.\
            tm_evaluator import (
                evaluate_topic_models
)

#########################################################################################
#                      Hybrid Summarization Engine (CLIMATEBart/C3)                     #
#########################################################################################
from nlp.\
        hybrid_text_summarization_engine.\
            ts_corpus_cleaner import (
                preprocess_text, # deployment
                preprocess_dataframe # training
)
from nlp.\
        hybrid_text_summarization_engine.\
            ts_feature_engineering import (
                feature_engineering_pipeline, # deployment
                feature_engineering_pipeline_dataframe # training
)
from nlp.\
        hybrid_text_summarization_engine.\
            ts_feature_visualizations import (
                visualize_tfidf,
                visualize_embeddings,
                visualize_topics,
                visualize_named_entities,
                visualize_keywords
)
from nlp.\
        hybrid_text_summarization_engine.\
            climate_corpus_condenser import (
                ClimateExtractiveModel, 
                CLIMATEBart, 
                ClimateCorpusCondenser as C3
)
from nlp.\
        hybrid_text_summarization_engine.\
            ts_evaluator import (
                BARTevaluator
)

#########################################################################################
#                          Sentiment Analysis Network (RoBi)                            #
#########################################################################################
from nlp.\
        sentiment_analysis_network.\
            hybrid_sentiment_network import (
                RoBi, 
                SlidingWindow, 
                train_RoBi, 
                save_RoBi, 
                load_RoBi, 
                predict_sentiment
)
from nlp.\
        sentiment_analysis_network.\
            robi_config import (
                SlidingWindowConfig, 
                RoBiConfig, 
                TrainConfig, 
                SaveLoadConfig, 
                PredictConfig,
                RoBiEvaluationConfig
)
from nlp.\
        sentiment_analysis_network.\
            robi_evaluator import (
                evaluate_robi, 
                evaluate_roberta, 
                evaluate_and_visualize
)

# Set random seeds for reproducibility, each package must be individually addressed to lock in randomized settings under the hood
random.seed(10) # standard python
np.random.seed(10) # numpy
tf.random.set_seed(10) # tensorflow
transformers.set_seed(10) # transformers
torch.manual_seed(10) # torch
if torch.cuda.is_available(): # GPU
    torch.cuda.manual_seed_all(10)

[nltk_data] Downloading package stopwords to C:\Users\Elijah
[nltk_data]     Taber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Elijah
[nltk_data]     Taber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Elijah Taber\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Load Training & Evaluation Data From SQLite Database

In [ ]:
conn = sqlite3.connect('tagged_climate_corpuses.csv')
cursor = conn.cursor()

print(f"Connected to SQLite database: {'tagged_climate_corpuses.csv'}")

In [ ]:
cursor.execute("""
SELECT name FROM sqlite_master 
WHERE type='table';
""")

tables = cursor.fetchall()
for table in tables:
    print(table[0])

#### Training Data

In [ ]:
def load_dataframe(conn):
    query = "SELECT * FROM TrainingData;"
    df = pd.read_sql_query(query, conn)
    print(f"Loaded Training DataFrame with {len(df)} records.")
    return df

training_corpus_df = load_dataframe(conn)
training_corpus_df.head()

#### Evaluation Data

In [ ]:
def load_dataframe(conn):
    query = "SELECT * FROM EvaluationData;"
    df = pd.read_sql_query(query, conn)
    print(f"Loaded Evaluation DataFrame with {len(df)} records.")
    return df

evaluation_corpus_df = load_dataframe(conn)
evaluation_corpus_df.head()

In [ ]:
# Close the database connection
conn.close()

## Normalizing

In [ ]:
# Create an instance of the TCNormalizer class
normalizer = TextNormalizer()

# Normalize the dataframe
normalized_df = normalizer.normalize_dataframe(training_corpus_df, 'corpus')
normalized_df.head()

#### Calculating The Full Token Count

In [ ]:
# Extract the corpus column as a list of strings
corpus_list = normalized_df['corpus'].tolist()

# Calculate the total token count
corpus_length = sum(len(document.split()) for document in corpus_list)

print(f"Total tokens in normalized corpus: {corpus_length}")

<center><h1>Climate Change Classification using a Convolutional Neural Network (C4NN)</h1></center>

#### 3D Classification Distribution

In [ ]:
sherical_visualizer = SphericalMap()
vectorizer = TfidfVectorizer(max_features=1000)

X = vectorizer.fit_transform(normalized_df['corpus']).toarray()

classification = normalized_df['classification'].tolist()
topic = normalized_df['topic'].tolist()

sherical_visualizer.create_visualization(X, classification, topic)

## Training The Tokenizer

#### Universal Sentence Encoder (USE)
Google's TensorFlow Universal Sentence Encoder is a pre-trained model that converts text into fixed-length numerical vectors, allowing neural networks to understand the semantic meaning of sentences. It is crucial in NLP models as it provides a way to represent variable-length text inputs as fixed-size embeddings, simplifying the task of processing text for downstream tasks like classification, clustering, or semantic similarity. In the tokenization phase of an NLP pipeline, the Universal Sentence Encoder is used to convert raw text into vector representations, which can then be fed into a neural network for further processing. This enables the network to learn from the semantic meanings of sentences, rather than just their individual words, which results in a more accurate and context-aware language understanding.

In [ ]:
# Download the Universal Sentence Encoder model and save it locally
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
tf.saved_model.save(use_model, "use_model")

#### Initialize & Fit the Tokenizer

In [ ]:
tokenizer = LexicalTokenizer(use_model_path='use_model')

# Train the tokenizer on the normalized dataframe
tokenizer.fit(
    normalized_df, 
    text_column='corpus',
    topic_column='topic',
    classification_column='classification', 
    sentiment_column='sentiment'
)

#### Save The Trained Tokenizer & Encoders

In [ ]:
tokenizer.save_tokenizer('tokenizer.pkl')

with open('topic_label_encoder.pkl', 'wb') as handle:
    pickle.dump(tokenizer.label_encoders['topic'], handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('classification_label_encoder.pkl', 'wb') as handle:
    pickle.dump(tokenizer.label_encoders['classification'], handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('sentiment_label_encoder.pkl', 'wb') as handle:
    pickle.dump(tokenizer.label_encoders['sentiment'], handle, protocol=pickle.HIGHEST_PROTOCOL)

## 100 Dimensional Embedding Matrix

In [ ]:
# Load in the embedding matrix creator with the trained tokenizer
embedding_matrix_creator = EmbeddingMatrix(
    tokenizer, # vocabulary created by the tokenizer
    embedding_dim = 100, # dimensions
    glove_path = 'GloVe/glove.6B.100d.txt' # pre-trained embeddings with 6 billion tokens
)

# Create the embedding matrix
embedding_matrix = embedding_matrix_creator.get_embedding_matrix()
for i in range(5):
    print(f"Row {i}: {embedding_matrix[i]}")

#### Save the Embedding Matrix

In [ ]:
np.save('embedding_matrix.npy', embedding_matrix)
print("Embedding matrix shape:", embedding_matrix.shape)

#### Tokenize & Encode The Training Data

In [ ]:
processed_data = tokenizer.preprocess_dataframe(
    normalized_df, 
    text_column='corpus', # pad
    topic_column='topic', # encode
    classification_column='classification', # encode
    sentiment_column='sentiment', # encode
    summary_column='summary' # pad
)

print("Text Embeddings Shape:", processed_data['text_embeddings'].shape)
print("Summary Embeddings Shape:", processed_data['summary_embeddings'].shape)

#### Data Preparation

In [ ]:
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

embedding_matrix = np.load('embedding_matrix.npy')

# Extract necessary data for training
X = processed_data['text']  # extract the padded text sequences
y = processed_data['classification']  # extract the encoded classification labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=10
)

# Once the training data is defined, further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size=0.2, 
    random_state=10
)

max_sequence_length = X.shape[1]
num_classes = len(np.unique(y))

print(f"Max sequence length: {max_sequence_length}")
print(f"Number of classes: {num_classes}")

#### Training and Tuning

In [ ]:
# Check for GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Create the CNNHyperModel
c4nn_hypermodel = C4NN(
    embedding_matrix, 
    max_sequence_length,
    num_classes
)

# Create the BayesianTuner
tuner = BayesianTuner(hypermodel=c4nn_hypermodel.architect)

# Search for the best model
tuner.search(
    X_train, 
    y_train, 
    X_val, 
    y_val,
)

# Extract and save the best classification model
best_model = tuner.get_best_model()
history = tuner.get_history()
tuner.save_best_model(best_model, path='best_c4nn_model.h5')

In [ ]:
loaded_model = BayesianTuner.load_best_model()

## Evaluation

In [ ]:
evaluator = C4NNEvaluator()

# Evaluate the model
loss, accuracy = evaluator.evaluate_model(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Get predictions
y_pred_proba = evaluator.get_predictions(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)

class_names = ['Research Article', 'News Article', 'Opinion Piece', 'Blog Post', 'Review Article', 
               'Policy Report', 'Educational Material', 'Data Report', 'Interviews and Profiles']

# Plot confusion matrix
evaluator.plot_confusion_matrix(y_test, y_pred, class_names)

# Plot ROC curve
evaluator.plot_roc_curve(y_test, y_pred_proba, class_names)

# Plot precision, recall, and F1-score
evaluator.plot_precision_recall_f1(y_test, y_pred, class_names)

evaluator.plot_training_history(history)

<center><h1>Topic Model</h1></center>

## CLIMATopic

In [ ]:
# Configurations
bertopic_config = BERTopicConfig()
save_config = SaveConfig()
vis_config = VisualizationConfig()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
docs = normalized_df['corpus'].tolist()

# Initialize CLIMATopic
climatopic_model = CLIMATopic(bertopic_config, save_config)
climatopic_model.to_device(device)

# Fit the model
topics, probs = climatopic_model.fit(docs)

print("Topics:", topics)
print("\nProbabilities:", probs)

In [ ]:
# Save the model
climatopic_model.save()

In [ ]:
# Load the model
climatopic_model.load(save_config.path)

In [ ]:
# Get topic information
topic_info = climatopic_model.topic_model.get_topic_info()
topic_info.head()

In [ ]:
embeddings = climatopic_model.embedding_model.encode(docs, show_progress_bar=True)
umap_model = create_umap()

visualize_umap(embeddings, umap_model)

In [ ]:
climatopic_model.topic_model.visualize_term_rank(log_scale=True)

In [ ]:
# Visualization of topics
visualize_topics(climatopic_model.topic_model)

# Visualization of heatmap
visualize_heatmap(climatopic_model.topic_model)

# Visualization of document datamap
visualize_document_datamap(climatopic_model.topic_model, docs, embeddings=embeddings, width=vis_config.width,
                           height=vis_config.height, title=vis_config.title, sub_title=vis_config.sub_title)

In [ ]:
# Get representations for a specific topic
topic_representation = climatopic_model.topic_model.get_topic(1, full=True)
print(topic_representation)

In [ ]:
# Predict topics for the new article
new_topics, new_probs = climatopic_model.topic_model.transform(new_article)

print("Predicted Topics:", new_topics)
print("Topic Probabilities:", new_probs)

## Evaluation

In [ ]:
# Load the pre-trained CLIMATopic model
climatopic_config = BERTopicConfig()
save_config = SaveConfig()
climatopic = CLIMATopic(climatopic_config, save_config)
climatopic.load("path/to/your/saved/climatopic/model")

bertopic = BERTopic()

# Load your evaluation articles (replace with your actual data loading code)
with open('path/to/your/evaluation/articles.txt', 'r') as f:
    evaluation_articles = f.readlines()

# Load your single complex document (replace with your actual data loading code)
with open('path/to/your/complex/document.txt', 'r') as f:
    complex_document = f.read()

# Generate topics for CLIMATopic (using transform)
climatopic_topics, climatopic_probs = climatopic.topic_model.transform(evaluation_articles)

# Fit and transform the standard BERTopic model
bertopic_topics, bertopic_probs = bertopic.fit_transform(evaluation_articles)

# Set up the evaluation configuration
eval_config = EvaluationConfig()

# Evaluate both models
results = evaluate_topic_models(
    climatopic.topic_model,
    bertopic,
    evaluation_articles,
    complex_document,
    eval_config
)

# The results, including visualizations, will be displayed automatically
results

<center><h1>Hybrid Text Summarization Engine</h1></center>

#### An In-Depth Analysis At Feature Engineering

In [ ]:
# Extract text from PDF to get a raw and very messy corpus
raw_corpus = pdf_text_extractor(
    "hybrid_text_summarization_engine/complex_pdf/IPCC_AR6_WGIII_TechnicalSummary.pdf", 
    start_page=7, 
    end_page=101
)
raw_corpus[:500] # first 500 characters

In [ ]:
cleaned_corpus = preprocess_text(raw_corpus)
cleaned_corpus[:500]

In [ ]:
# Create a Doc object by applying the NLP model to the cleaned corpus
doc = nlp(cleaned_corpus)

In [ ]:
# Extract sentences from the Doc object and store them in a list
sentences = [sent.text for sent in doc.sents]

In [ ]:
# Perform feature engineering on the extracted sentences
features = feature_engineering_pipeline(sentences)

#### Term Frequency-Inverse Document Frequency

In [ ]:
print("Top 10 TF-IDF Scores:")
counter = 0
for term, score in features['tfidf'].items():
    print(f"{term}: {score}")
    counter += 1
    if counter == 10:
        break

In [ ]:
visualize_tfidf(features['tfidf'])

#### Sentence Embeddings

In [ ]:
print("First 10 Sentence Embeddings:")
for index, embedding in enumerate(features['embeddings']):
    print(embedding)
    if index == 9:
        break

In [ ]:
visualize_embeddings(features['embeddings'])

#### Topic Clustering

In [ ]:
print("LDA Topics:")
print(features['topics'])

In [ ]:
visualize_topics(features['topics'])

#### Named Entity Recognition

In [ ]:
print("First 10 Named Entities:")
for index, entity in enumerate(features['entities']):
    print(f"Entity: {entity[0]}, Type: {entity[1]}")
    if index == 9:
        break

In [ ]:
visualize_named_entities(features['entities'])

#### Keywords

In [ ]:
print("First 10 Keywords:")
for index, keyword in enumerate(features['keywords']):
    print(keyword)
    if index == 9:
        break

In [ ]:
visualize_keywords(features['keywords'])

## Climate Corpus Condenser (C3)
#### A Climate Change Tuned BART Model

In [ ]:
# 1. Corpus cleaning
cleaned_df = preprocess_dataframe(training_corpus_df, 'corpus')

# 2. Feature engineering
features = feature_engineering_pipeline_dataframe(cleaned_df, 'corpus')

articles = cleaned_df['corpus'].tolist()
summaries = cleaned_df['summary'].tolist()

# 3. Creating the extraction model with the features
extractive_model = ClimateExtractiveModel(top_n=10)

# 4. Training the BART model with the extraction model to create a hybrid text summarization engine
abstractive_model = CLIMATEBart()

# 5. Creating the hybrid model (ClimateCorpusCondenser)
hybrid_model = C3(abstractive_model)

# 6. Train the hybrid model
hybrid_model.train(articles, summaries, extractive_model, features)

In [ ]:
# 7. Save the hybrid model
hybrid_model.save('C3_model')

In [ ]:
# 8. To use the model for summarization later:
c3_model = C3(CLIMATEBart())
c3_model.load('C3_model')

In [ ]:
# Example usage
new_article = "Your climate change article text here..."
cleaned_article = preprocess_text(new_article)
summary = c3_model.summarize(cleaned_article)
print(summary)

## Standard BART Model (No Climate Corpus Training)

In [ ]:
bart_model = BartModel.from_pretrained('facebook/bart-large')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

## Evaluation

In [ ]:
evaluation_df = pd.read_csv('evaluation_corpuses.csv')
eval_articles = evaluation_df['corpus'].tolist()
reference_summaries = evaluation_df['summary'].tolist()

In [ ]:
c3_model = C3(CLIMATEBart())
c3_model.load('C3_model')

In [ ]:
evaluator = BARTevaluator(c3_model, bart_model)

c3_results, bart_results = evaluator.compare_models(eval_articles, reference_summaries)

print("C3 Model Results:")
print(c3_results)
print("\nBART Model Results:")
print(bart_results)

In [ ]:
evaluator.visualize_results(c3_results, bart_results)

In [ ]:
c3_summaries = [c3_model.summarize(article) for article in tqdm(eval_articles, desc="Generating C3 summaries")]

In [ ]:
evaluator.generate_summary_wordcloud(c3_summaries)

In [ ]:
evaluator.analyze_topic_coverage(eval_articles, c3_summaries)

<center><h1>Hybrid Sentiment Analysis Network</h1></center>

## RoBi

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Load in training corpuses
texts = normalized_df['corpus'].tolist()

# Encode sentiment labels to numerical labels for training
labels = normalized_df['sentiment'].map({
    'Very Negative': 0, 
    'Negative': 1, 
    'Neutral': 2, 
    'Positive': 3, 
    'Very Positive': 4
}).tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, 
    labels, 
    test_size=0.2, 
    random_state=10
)

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-large')

train_dataset = SlidingWindow(
    train_texts, 
    train_labels, 
    roberta_tokenizer, 
    config=SlidingWindowConfig()
)
val_dataset = SlidingWindow(
    val_texts, 
    val_labels, 
    roberta_tokenizer, 
    config=SlidingWindowConfig()
)

# Create data loaders
batch_size = 64
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True
)
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=batch_size
)

In [ ]:
robi = RoBi(config=RoBiConfig())

robi_train_config = TrainConfig(device=device)
trained_robi = train_RoBi(
    robi, 
    train_dataloader, 
    val_dataloader, 
    robi_train_config
)

In [ ]:
robi_save_config = SaveLoadConfig()
save_RoBi(
    trained_robi, 
    roberta_tokenizer, 
    robi_save_config
)
print(f"Model and tokenizer saved successfully to {save_config.path}")

In [ ]:
load_config = SaveLoadConfig()  # Use default path from config
loaded_model, loaded_tokenizer = load_RoBi(config=load_config, robi_config=RoBiConfig())
print(f"Model and tokenizer loaded successfully from {load_config.path}")

The model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. We need to turn them off during model evaluation, and .eval() will do it for us. In addition, the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation.

In [ ]:
# Predict sentiment for new text
new_text = "This is a sample text for sentiment analysis."
predict_config = PredictConfig()
sentiment = predict_sentiment(
    loaded_model, 
    loaded_tokenizer, 
    new_text, 
    device, 
    predict_config
)

print(f"Predicted sentiment: {sentiment}")
# 0: Very Negative, 1: Negative, 2: Neutral, 3: Positive, 4: Very Positive

## Evaluation

In [ ]:
eval_df = pd.read_csv('path_to_your_evaluation_data.csv')
print(eval_df.shape)
eval_df.head()

<center><h4>RoBi</h4></center>

In [ ]:
robi_eval_config = RoBiEvaluationConfig()
robi_predict_config = PredictConfig()

print("Evaluating RoBi model...")

robi_model, robi_tokenizer = load_RoBi()
robi_model.to(robi_eval_config.device)

robi_preds, robi_true = evaluate_robi(
    robi_model, 
    robi_tokenizer, 
    eval_df, 
    robi_eval_config.device, 
    robi_eval_config, 
    robi_predict_config
)

In [ ]:
robi_proba = []
for _, row in eval_df.iterrows():
    outputs = robi_model(
        robi_tokenizer(
            row['corpus'], 
            return_tensors="pt", 
            truncation=True, 
            max_length=512, 
            padding="max_length"
        ).to(robi_eval_config.device))

    probs = torch.softmax(outputs, dim=1)
    robi_proba.append(probs.cpu().detach().numpy()[0])
    
robi_proba = np.array(robi_proba)

evaluate_and_visualize(
    robi_preds, 
    robi_true, 
    robi_proba, 
    "RoBi", 
    robi_eval_config.class_names
)

<center><h4>Standard RoBERTa</h4></center>

In [ ]:
print("\nEvaluating standard RoBERTa model...")

# Load the pre-trained RoBERTa model and its tokenizer
roberta_model = RobertaModel.from_pretrained(robi_eval_config.roberta_model_name)
roberta_tokenizer = RobertaTokenizer.from_pretrained(robi_eval_config.roberta_model_name)
roberta_model.to(robi_eval_config.device)

# Evaluate RoBERTa
roberta_preds, roberta_true = evaluate_roberta(
    roberta_model, 
    roberta_tokenizer, 
    eval_df, 
    robi_eval_config.device, 
    robi_eval_config
)

In [ ]:
# Calculate probabilities for ROC curve (for RoBERTa)
roberta_proba = []
for _, row in eval_df.iterrows():
    inputs = roberta_tokenizer(
        row['corpus'], 
        return_tensors="pt", 
        truncation=True, 
        max_length=512, 
        padding="max_length"
    ).to(robi_eval_config.device)
    outputs = roberta_model(**inputs).last_hidden_state.mean(dim=1)
    logits = torch.nn.Linear(
        roberta_model.config.hidden_size, 
        robi_eval_config.num_classes
    ).to(robi_eval_config.device)(outputs)
    probs = torch.softmax(logits, dim=1)
    roberta_proba.append(probs.cpu().detach().numpy()[0])

roberta_proba = np.array(roberta_proba)

# Visualize RoBERTa results
evaluate_and_visualize(
    roberta_preds, 
    roberta_true, 
    roberta_proba, 
    "RoBERTa", 
    robi_eval_config.class_names
)